In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import print_function

import datetime
import pandas as pd
import numpy as np
from dateutil import parser as dtparser

from utils.datafetch import *
from utils.vectorized_funs import *
from utils.datapipe import *
from utils.kerasutil import *
from utils import scenarioa
from FintechCapstone import FinCapstone

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1070 (CNMeM is disabled, cuDNN 5105)


In [ ]:
%matplotlib inline

plt.style.use('seaborn-whitegrid')
date_from = '2009-01-01'
date_to = '2017-03-01'
timespan = {
    "short_term": [1, 5]
    ,"medium_term": [40]
    ,"long_term": [90]
}

trial = FinCapstone(ticker_list=["GLW","HIMX","GLUU"], ticker_list_samplesize=4, timespan=timespan, date_from=date_from)
#trial = FinCapstone(ticker_list=["ZHNE","BSFT","AERO","MATR","ITMSF","FIT","TIPT","OLBK","QADA","ATTU","LGF","STX","TPCS","GSAT","ZNGA","GLW","HIMX","YHOO", "NWSA","JAKK","GLUU","AMZN", "AAPL", "EBAY", "GOOG", "DIS", "NFLX", "EA", "TWTR", "FB", "TTWO", "PXLW", "UBI"], ticker_list_samplesize=4, timespan=timespan, timespan_ab=timespan_ab, date_from=date_from)
#trial = FinCapstone(ticker_list_samplesize=200, timespan=timespan, timespan_ab=timespan_ab, date_from=date_from)

trial.run_initial_dataload()
trial.feature_engineering(feature_set="scenarioa")

(1/3) Recv. and Stored GLW
(2/3) Recv. and Stored HIMX
(3/3) Recv. and Stored GLUU
Took 0:00:02.656143


 - GLW - 



 - HIMX - 



In [ ]:
X = trial.load_scenarioa_features("GLUU", True)
y = trial.load_scenarioa_labels("GLUU", True)

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
features_df = trial.load_scenarioa_features("GLUU", parseDate=True)
features_df.set_index("Date", inplace=True)

labels_df = trial.load_scenarioa_labels("GLUU", parseDate=True)
labels_df.set_index("Date", inplace=True)

model = scenarioa.create_model()
X_train, y_train, X_test, y_test = scenarioa.prepare_problemspace("GLUU", trial.valid_ticker_list(), trial.train_from, trial.train_until, trial.test_from)
X_train


In [ ]:
X_train, y_train, X_test, y_test = scenarioa.prepare_problemspace("GLUU", trial.valid_ticker_list(), trial.train_from, trial.train_until, trial.test_from, True)

In [ ]:
X_train

In [ ]:
y_train.head()

In [ ]:
X_test

In [ ]:
y_test.head()

In [ ]:
X_train, y_train, X_test, y_test = scenarioa.prepare_problemspace("GLUU", trial.valid_ticker_list(), trial.train_from, trial.train_until, trial.test_from, "numpy")

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
y_test.shape

In [126]:
model = scenarioa.create_model()

MemoryError: ('Error allocating 226492416 bytes of device memory (out of memory).', "you might consider using 'theano.shared(..., borrow=True)'")

In [ ]:
#model.summary()

In [96]:
X_train, y_train, X_test, y_test = scenarioa.prepare_problemspace("GLUU", trial.valid_ticker_list(), trial.train_from, trial.train_until, trial.test_from, "pandas")

In [97]:
scenarioa.fit(model, X_train, y_train)

Epoch 1/2
1510/1510 [==============================] - 4s - loss: nan     
Epoch 2/2
1510/1510 [==============================] - 4s - loss: nan     


In [77]:
X_train

<class 'pandas.core.panel.Panel'>
Dimensions: 1510 (items) x 29 (major_axis) x 3 (minor_axis)
Items axis: 2010-01-04 00:00:00 to 2015-12-31 00:00:00
Major_axis axis: Close to OBV
Minor_axis axis: GLUU to HIMX

In [78]:
y_train

,RETURN_1,RETURN_30,RETURN_60,RETURN_200
Date,,,,
2010-01-04,0.049587,-0.132231,-0.190083,0.363636
2010-01-05,0.000000,-0.181102,-0.212598,0.322835
2010-01-06,0.031496,-0.188976,-0.212598,0.393701
2010-01-07,0.106870,-0.259542,-0.206107,0.343511
2010-01-08,-0.013793,-0.365517,-0.275862,0.151724
2010-01-11,-0.034965,-0.349650,-0.307692,0.188811
2010-01-12,-0.014493,-0.333333,-0.275362,0.231884
2010-01-13,0.051471,-0.323529,-0.139706,0.257353
2010-01-14,0.006993,-0.356643,-0.167832,0.195804
